## Scraping job listing details in Indeed

In [79]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [80]:
def getRecord(tag):
    # get individual result of each card
    url = 'https://jp.indeed.com' + tag.h2.a.get('href')
    title = tag.h2.a.get('title')
    company = tag.find('span', class_='company').text.strip()
    location = tag.find('div', class_='recJobLoc').get('data-rc-loc')
    summary = tag.find('div', class_='summary').text.strip()
    try:
        salary = tag.find_all('span', class_='salaryText')[0].text.strip()
    except:
        salary = 'not listed'
    
    record = (title, company, location, salary, summary, url)
    return record


In [83]:
def main(position, location):
    """Generate url from position and location"""
    base = 'https://jp.indeed.com/jobs?q={}&l={}'
    position = position.replace(' ', '+')
    location = location.replace(' ', '+')
    url = base.format(position, location)
    records = list()
    
    while True:
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')
        tags = soup.find_all('div', class_='jobsearch-SerpJobCard')

        for tag in tags:
            record = getRecord(tag)
            records.append(record)
        
        try:
            url = 'https://jp.indeed.com' + soup.find('a', {'aria-label': '次へ'}).get('href') #get url of next page
        except:
            break
    
    #export result to csv file
    df = pd.DataFrame(records, columns=['Title', 'Company', 'Location', 'Salary', 'Summary', 'URL'])
    df.to_csv('results.csv', encoding='utf_8_sig')

In [84]:
main('data analyst', '東京都')